# Background Solution

## Preamble

In [1]:
%load_ext autoreload

In [2]:
%autoreload

import matplotlib.pyplot as plt

import sys
sys.path.append('..')

import jax.numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

# Load plot settings

from plot_params import params
pylab.rcParams.update(params)

cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

%matplotlib inline

In this notebook, we examine the Standard BBN (SBBN) background solution, and demonstrate how to calculate several important quantities. 

The class that computes all background quantities is `linx.background.BackgroundModel`. Additional thermodynamic quantities can be found in `linx.thermo`. 

In [3]:
from linx.background import BackgroundModel
import linx.thermo as th
import linx.const as cn 

The default SBBN background can be computed by creating an instance of `BackgroundModel`, and then calling the class, specifying the number of extra relativistic degrees of freedom, which for SBBN is 0. 

The output is a tuple of arrays, all of the same length, representing 
time (in s), scale factor, photon energy density (in MeV^4), the energy density of one neutrino species (in MeV^4), the energy density of the relativistic degrees of freedom (zero for SBBN), the pressure of the relativistic degrees of freedom (also zero for SBBN), and $N_\text{eff}$, defined as

$$
    N_\text{eff} = \frac{8}{7} \left( \frac{11}{4} \right)^{4/3} \frac{\rho_\text{tot} - \rho_\gamma}{\rho_\gamma}
$$

In [4]:
# Initialize the class. 
default = BackgroundModel() 

# Call the class. 
t_vec, a_vec, rho_g_vec, rho_nu_vec, _, _, Neff_vec = default(jnp.asarray(0.)) 

`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


Note we pass the argument to `BackgroundModel()` in wrapped in `jnp.asarray()`; this helps prevent recompilation of equinox modules.

Let's make a plot of $a T_\gamma$ and $a T_\nu$ as a function of $T_\gamma$. We can see the entropy dump from $e^+e^-$ annihilation starting at temperatures near the mass of the electron, causing the $a T_\gamma$ to rise, reaching the value it has today. Meanwhile, neutrinos decoupled at 2 MeV, and did not experience significant heating from $e^+e^-$ annihilation. 

In [5]:
# Get the photon temperature from its energy density
T_gamma = th.T_g(rho_g_vec)
T_nu    = th.T_nu(rho_nu_vec) 

plt.figure() 

plt.plot(T_gamma, a_vec * T_gamma / cn.kB, label=r'$T_\gamma$') 
plt.plot(T_gamma, a_vec * T_nu / cn.kB, label=r'$T_\nu$') 
plt.axhline(cn.T0CMB / cn.kB, xmin=1e-4, xmax=1e2, ls='--', color='k', label=r'$T_{\mathrm{CMB},0}$')

plt.xscale('log') 

plt.xlabel(r'$T_\gamma$ [MeV]')
plt.ylabel(r'Temperature [K]')

plt.legend()

Error in callback <function _draw_all_if_interactive at 0x7980ca9b8ea0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 443.077x360 with 1 Axes>

Included in `BackgroundModel` are many corrections that ensure sub-1% accuracy in the background quantities. We now illustrate the relative importance of these corrections by solving for the background quantities under different assumptions. 

Note that these next evaluations of `BackgroundModel` will still recompile each time.  This is because `BackgroundModel.__call__` has only been compiled for one set of options--each new set of options requires recompilation.

In [ ]:
##### Neutrinos decouple before e+e- annihilation #####

# No QED corrections to EM plasma properties. 
decoupled_noQED = BackgroundModel(decoupled=True, LO=False, NLO=False) 
decoupled_noQED_res = decoupled_noQED(jnp.asarray(0.))

# Same as above, but with leading order QED corrections. 
decoupled_LO = BackgroundModel(decoupled=True, LO=True, NLO=False) 
decoupled_LO_res = decoupled_LO(jnp.asarray(0.))

# Same as above, but with next-to-leading order QED corrections. 
decoupled_NLO = BackgroundModel(decoupled=True, LO=True, NLO=True) 
decoupled_NLO_res = decoupled_NLO(jnp.asarray(0.))

##### Neutrino decoupling turned on, massless electrons. #####

# Maxwell-Boltzmann neutrino distribution, no QED corrections. 
MB_zerome_noQED = BackgroundModel(
    use_FD=False, collision_me = False, LO=False, NLO=False
) 
MB_zerome_noQED_res = MB_zerome_noQED(jnp.asarray(0.))

# Maxwell-Boltzmann neutrino distribution, all QED corrections. 
MB_zerome_NLO = BackgroundModel(use_FD=False, collision_me = False) 
MB_zerome_NLO_res = MB_zerome_NLO(jnp.asarray(0.))

# Fermi-Dirac neutrino distribution, no QED corrections. 
FD_zerome_noQED = BackgroundModel(collision_me=False, LO=False, NLO=False)
FD_zerome_noQED_res = FD_zerome_noQED(jnp.asarray(0.))

# Fermi-Dirac neutrino distribution, no QED corrections. 
FD_zerome_NLO = BackgroundModel(collision_me=False)
FD_zerome_NLO_res = FD_zerome_NLO(jnp.asarray(0.))

##### Neutrino decoupling turned on, Fermi-Dirac distribution, massive electrons. #####

# No QED corrections.
FD_me_noQED = BackgroundModel(LO=False, NLO=False) 
FD_me_noQED_res = FD_me_noQED(jnp.asarray(0.)) 

# Leading order QED corrections. 
FD_me_LO = BackgroundModel(NLO=False)
FD_me_LO_res = FD_me_LO(jnp.asarray(0.))

# All QED corrections. 
FD_me_NLO = BackgroundModel()
FD_me_NLO_res = FD_me_NLO(jnp.asarray(0.))

`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


...

/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


In [ ]:
# Compile results into a single array
res = jnp.array([
    decoupled_noQED_res, decoupled_LO_res, decoupled_NLO_res, 
    MB_zerome_noQED_res, MB_zerome_NLO_res, 
    FD_zerome_noQED_res, FD_zerome_NLO_res, 
    FD_me_noQED_res, FD_me_LO_res, FD_me_NLO_res
])


In [ ]:
Neff_ID = res[0, -1, -1]

print('Final ratio of temperatures T_g/T_nu and Neff: ')

print(
    'Instantaneous decoupling:            T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[0, 2, -1]) / th.T_nu(res[0, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[0, -1, -1]), 
    '% Diff.: {:1.2f}'.format((res[0, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'Instantaneous decoupling + LO-QED:   T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[1, 2, -1]) / th.T_nu(res[1, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[1, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[1, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'Instantaneous decoupling + NLO-QED:  T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[2, 2, -1]) / th.T_nu(res[2, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[2, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[2, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'MB collision term:                   T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[3, 2, -1]) / th.T_nu(res[3, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[3, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[3, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'MB collision term + NLO-QED:         T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[4, 2, -1]) / th.T_nu(res[4, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[4, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[4, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'FD collision term:                   T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[5, 2, -1]) / th.T_nu(res[5, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[5, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[5, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'FD collision term + NLO-QED:         T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[6, 2, -1]) / th.T_nu(res[6, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[6, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[6, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'FD+me collision term:                T_g/T_nu: {:1.4f}'.format(
        th.T_g(res[7, 2, -1]) / th.T_nu(res[7, 3, -1])
    ), 
    '  Neff: {:1.3f}'.format(res[7, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[7, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'FD+me collision term + LO-QED:       T_g/T_nu: {:1.5f}'.format(
        th.T_g(res[8, 2, -1]) / th.T_nu(res[8, 3, -1])
    ), 
    ' Neff: {:1.3f}'.format(res[8, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[8, -1, -1] - Neff_ID) / Neff_ID * 100)
)
print(
    'FD+me collision term + NLO-QED:      T_g/T_nu: {:1.5f}'.format(
        th.T_g(res[9, 2, -1]) / th.T_nu(res[9, 3, -1])
    ), 
    ' Neff: {:1.3f}'.format(res[9, -1, -1]), 
    
    '% Diff.: {:1.2f}'.format((res[9, -1, -1] - Neff_ID) / Neff_ID * 100)
)

These results can be compared to Escudero arXiv:2001.04466 Table 1, which our background calculation is based on: the agreement is excellent. 